In [45]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET

## Read local status of packages

This table is where we assign core, signature, local keywords, and other categorizations

In [46]:
localstatus = pd.read_csv('edi_package_localstatus.csv')
print(len(localstatus))
localstatus.head()

211


,studyid,pkgid,pubdate,title,authors,old_core,core_sig_2020prop,top10_2020prop,top10doi_2020prop,corearea
0,1,100,2019,High resolution shrub cover raster maps of the...,"Ji, Wenjie; Hanan, Niall P.;",NaN,signature,NaN,NaN,Inorganic matter cycling
1,1,210001001,2011,Hydrology natural runoff plots plant cover,"Ward, Tim;",NaN,NaN,NaN,NaN,Inorganic matter cycling
2,1,210001002,2019,Rainfall runoff and sediment deposition from 2...,"Ward, Tim;",core,core,NaN,NaN,Inorganic matter cycling
3,1,210001003,2011,Hydrology plot surface runoff water chemistry,"Ward, Tim;",NaN,NaN,NaN,NaN,Inorganic matter cycling
4,2,210002001,2019,Graduated rain gauge (GRG) precipitation obser...,"Huenneke, Laura; Anderson, John;",core,core,1.0,https://doi.org/10.1007/s00442-013-2837-y,Disturbance


### Make changes if needed and resave

Add keywords, alter core/signature, etc

In [47]:
#Assign core areas by study id
localstatus['corearea'] = 'None'
lulc = localstatus.studyid.isin(['1','119','120','392','472'])
localstatus.loc[lulc, 'corearea'] = 'Land Use and Land Cover Change'

inorganic = localstatus.studyid.isin(['001','012','013','128','228','288','338','365','368','386',
                                      '394','395'])
localstatus.loc[inorganic, 'corearea'] = 'Inorganic matter cycling'

organic = localstatus.studyid.isin(['301','327'])
localstatus.loc[organic, 'corearea'] = 'Organic matter cycling'

pops = localstatus.studyid.isin(['007','008','010','351','370','369','385','412','520'])
localstatus.loc[pops, 'corearea'] = 'Populations'

disturbance = localstatus.studyid.isin(['002','122','126','127','278','308','328','379','380','407',
                                        '425','437','461'])
localstatus.loc[disturbance, 'corearea'] = 'Disturbance'

prod = localstatus.studyid.isin(['011','287','409'])
localstatus.loc[prod, 'corearea'] = 'Primary production'

In [48]:
# Assign core areas by pkgid (studies 015, 086, 121, 262)
lulc = localstatus.pkgid.isin([''])
localstatus.loc[lulc, 'corearea'] = 'Land Use and Land Cover Change'

inorganic = localstatus.pkgid.isin(['210015002','210015003'])
localstatus.loc[inorganic, 'corearea'] = 'Inorganic matter cycling'

organic = localstatus.pkgid.isin(['210086004'])
localstatus.loc[organic, 'corearea'] = 'Organic matter cycling'

pops = localstatus.pkgid.isin(['210086001','210086002','210086003','210086005','210086006','210086008','210086009',
                      '210121001','210121006','210262001','210262008','210262010'])
localstatus.loc[pops, 'corearea'] = 'Populations'

disturbance = localstatus.pkgid.isin(['210086007','210121005','210121008'])
localstatus.loc[disturbance, 'corearea'] = 'Disturbance'

prod = localstatus.pkgid.isin(['210015001','210262004','210262005'])
localstatus.loc[prod, 'corearea'] = 'Primary production'

In [49]:
localstatus.loc[localstatus.corearea=='None',:]

,studyid,pkgid,pubdate,title,authors,old_core,core_sig_2020prop,top10_2020prop,top10doi_2020prop,corearea


In [50]:
localstatus_out = localstatus.copy()
localstatus_out.to_csv('edi_package_localstatus.csv', index=False)

## Get table from EDI

This lists all data packages at EDI under the knb-lter-jrn scope.

In [51]:
# Get info for all packages under our scope on EDI
params = (
    ('defType', 'edismax'),
    ('q', '*'),
    ('fq', 'scope:knb-lter-jrn'),
    ('fl', 'packageid,title,pubdate,keyword,author,begindate,enddate,doi'),
    ('sort','packageid'),
    ('rows','500') # Increase as our # of packages grows
)

response = requests.get('https://pasta.lternet.edu/package/search/eml', params=params)

In [52]:
response.text[0:1000]

"<resultset numFound='211' start='0' rows='500'>\n    <document>\n        <packageid>knb-lter-jrn.210001001.61</packageid>\n        <title>Hydrology natural runoff plots plant cover</title>\n        <pubdate>2011</pubdate>\n        <keywords>\n            <keyword>LTER</keyword>\n            <keyword>LTAR</keyword>\n            <keyword>Populations</keyword>\n            <keyword>Plant</keyword>\n            <keyword>hydrology</keyword>\n            <keyword>plant cover</keyword>\n            <keyword>completed</keyword>\n            <keyword>Hydrology</keyword>\n        </keywords>\n        <authors>\n            <author>Ward, Tim</author>\n        </authors>\n        <begindate>1982-09-09</begindate>\n        <enddate>1994-10-15</enddate>\n        <doi>doi:10.6073/pasta/1c55425053e9c8dc018a8ac4afce72b9</doi>\n    </document>\n    <document>\n        <packageid>knb-lter-jrn.210001003.78</packageid>\n        <title>Hydrology plot surface runoff water chemistry</title>\n        <pubdate

In [53]:
root = ET.fromstring(response.text)

In [54]:
# Create a dataframe from the entire record in root
df = pd.DataFrame({'packageid':[pid.text for pid in root.iter('packageid')],
                   'title':[title.text for title in root.iter('title')],
                   'pubdate':[pubdate.text for pubdate in root.iter('pubdate')],
                   'keyword':[';'.join([kw.text for kw in kws.iter('keyword')]) 
                              for kws in root.iter('keywords')],
                   'creators':[';'.join(list(reversed([auth.text for auth in auths.iter('author')]))) 
                              for auths in root.iter('authors')], #Order reversed from EML
                   #'creators':[rp.text for rp in root.iter('responsibleParties')],
                   'begindate':[sd.text for sd in root.iter('begindate')],
                   'enddate':[ed.text for ed in root.iter('enddate')],
                   'doi':[doi.text for doi in root.iter('doi')]})

In [55]:
print(len(df))
df.head()

211


,packageid,title,pubdate,keyword,creators,begindate,enddate,doi
0,knb-lter-jrn.210001001.61,Hydrology natural runoff plots plant cover,2011,LTER;LTAR;Populations;Plant;hydrology;plant co...,"Ward, Tim",1982-09-09,1994-10-15,doi:10.6073/pasta/1c55425053e9c8dc018a8ac4afce...
1,knb-lter-jrn.210001003.78,Hydrology plot surface runoff water chemistry,2011,LTAR;LTER;Inorganic Nutrients;Hydrology;hydrol...,"Ward, Tim",1988-06-27,1990-09-11,doi:10.6073/pasta/8bb9a06d8be9f573d30d462f7959...
2,knb-lter-jrn.210121001.47,Biodiversity plots vegetation transects,2012,LTER;LTAR;Populations;Plant;completed,"Anderson, John",None,None,doi:10.6073/pasta/1bd53fa81dc2e00e6d25ded39974...
3,knb-lter-jrn.210327001.1,Greenfall litter decomposition of three common...,2018,carbon;nitrogen;Jornada Basin LTER;JRN;decompo...,"Smith, Jane G.",2010-09-15,2012-09-23,doi:10.6073/pasta/06db7b16fe62bcce4c43fd9ddbe4...
4,knb-lter-jrn.210015001.46,Transect Vegetation Biomass,2011,LTAR;LTER;Populations;Plant;forbs;transects;bi...,"Anderson, John",None,None,doi:10.6073/pasta/6d8042aebd60fca10646e912eeb4...


In [56]:
df[df.packageid=='knb-lter-jrn.100.1']

,packageid,title,pubdate,keyword,creators,begindate,enddate,doi
71,knb-lter-jrn.100.1,High resolution shrub cover raster maps of the...,2019,canopy cover;land surface properties;plant cov...,"Hanan, Niall P.;Ji, Wenjie",2011-01-01,2011-12-31,doi:10.6073/pasta/2bbee949ad08c7feb1d5cec6570b...


In [57]:
pkgid = df.packageid.str.split('.', expand=True).loc[:,1]
df['pkgid'] = pkgid.astype(int)
df['studyid'] = pkgid.astype(str).str[0:-2]
df.loc[pkgid.astype(int) > 210000000, 'studyid'] = pkgid.loc[pkgid.astype(int) > 210000000].str[-6:-3]
df.head()

,packageid,title,pubdate,keyword,creators,begindate,enddate,doi,pkgid,studyid
0,knb-lter-jrn.210001001.61,Hydrology natural runoff plots plant cover,2011,LTER;LTAR;Populations;Plant;hydrology;plant co...,"Ward, Tim",1982-09-09,1994-10-15,doi:10.6073/pasta/1c55425053e9c8dc018a8ac4afce...,210001001,001
1,knb-lter-jrn.210001003.78,Hydrology plot surface runoff water chemistry,2011,LTAR;LTER;Inorganic Nutrients;Hydrology;hydrol...,"Ward, Tim",1988-06-27,1990-09-11,doi:10.6073/pasta/8bb9a06d8be9f573d30d462f7959...,210001003,001
2,knb-lter-jrn.210121001.47,Biodiversity plots vegetation transects,2012,LTER;LTAR;Populations;Plant;completed,"Anderson, John",None,None,doi:10.6073/pasta/1bd53fa81dc2e00e6d25ded39974...,210121001,121
3,knb-lter-jrn.210327001.1,Greenfall litter decomposition of three common...,2018,carbon;nitrogen;Jornada Basin LTER;JRN;decompo...,"Smith, Jane G.",2010-09-15,2012-09-23,doi:10.6073/pasta/06db7b16fe62bcce4c43fd9ddbe4...,210327001,327
4,knb-lter-jrn.210015001.46,Transect Vegetation Biomass,2011,LTAR;LTER;Populations;Plant;forbs;transects;bi...,"Anderson, John",None,None,doi:10.6073/pasta/6d8042aebd60fca10646e912eeb4...,210015001,015


## Make Table1


In [60]:
# Merge in core designations and rename them to "signature"
t1 = pd.merge(df, localstatus.loc[:,['pkgid','core_sig_2020prop','corearea',
                                     'top10_2020prop']], on='pkgid')
t1.core_sig_2020prop.replace('core', 'signature',inplace=True)
print(len(t1))

# Replace or add whitespace for a couple columns
t1.title = t1.title.str.replace('\n +', ' ')
t1.creators = t1.creators.str.replace(';', '; ')
# Trim version # from package id
t1.packageid = t1.packageid.map(lambda x: x.rsplit('.',1)[0])
# Add a date range
t1['drange'] = t1.begindate + ' to ' + t1.enddate
# Replace core areas with abbreviations
t1.corearea.replace('Disturbance', 'D', inplace=True)
t1.corearea.replace('Land Use and Land Cover Change', 'Lu', inplace=True)
t1.corearea.replace('Primary production', 'Pp', inplace=True)
t1.corearea.replace('Populations', 'Po', inplace=True)
t1.corearea.replace('Inorganic matter cycling', 'Im', inplace=True)
t1.corearea.replace('Organic matter cycling', 'Om', inplace=True)
# Add sorting order column for corearea
t1['corearea_sort'] = 0
t1.corearea_sort.loc[t1.corearea=='Pp'] = 1
t1.corearea_sort.loc[t1.corearea=='Om'] = 2
t1.corearea_sort.loc[t1.corearea=='D'] = 3
t1.corearea_sort.loc[t1.corearea=='Im'] = 4
t1.corearea_sort.loc[t1.corearea=='Po'] = 5
t1.corearea_sort.loc[t1.corearea=='Lu'] = 6
# Sort values
t1 = t1.sort_values(['core_sig_2020prop', 'corearea_sort', 'pkgid'],
                   ascending=['True', 'True', 'True'])

# Fill in NA values with empty string
t1['core_sig_2020prop'].fillna('', inplace=True)
t1['top10_2020prop'].fillna('', inplace=True)

t1.index = t1.loc[:,'pkgid']
t1.head()

211


,packageid,title,pubdate,keyword,creators,begindate,enddate,doi,pkgid,studyid,core_sig_2020prop,corearea,top10_2020prop,drange,corearea_sort
pkgid,,,,,,,,,,,,,,,
210011001,knb-lter-jrn.210011001,Seasonal aboveground plant biomass estimates a...,2019,net primary production;aboveground production;...,"Huenneke, Laura; Peters, Debra",1989-05-01,2012-09-25,doi:10.6073/pasta/5e11b55bec248c720809f1d804e5...,210011001,011,signature,Pp,1,1989-05-01 to 2012-09-25,1
210011002,knb-lter-jrn.210011002,Seasonal non-destructive vegetation measuremen...,2019,net primary production;aboveground production;...,"Huenneke, Laura; Peters, Debra",1989-05-01,2018-09-14,doi:10.6073/pasta/9053217dd75ad98e495bef87b459...,210011002,011,signature,Pp,1,1989-05-01 to 2018-09-14,1
210011003,knb-lter-jrn.210011003,Annual mean estimates of aboveground net prima...,2020,net primary production;aboveground production;...,"Huenneke, Laura; Peters, Debra",1990-01-01,2018-12-31,doi:10.6073/pasta/18dad6748af96c98b72cea3436bf...,210011003,011,signature,Pp,1,1990-01-01 to 2018-12-31,1
210011004,knb-lter-jrn.210011004,Seasonal reference harvest measurements of veg...,2019,net primary production;aboveground production;...,"Huenneke, Laura; Peters, Debra",1989-04-25,2018-09-10,doi:10.6073/pasta/bb54f668824c165818255dfa9e1f...,210011004,011,signature,Pp,1,1989-04-25 to 2018-09-10,1
210011005,knb-lter-jrn.210011005,Annual ground-based photographs taken at 15 ne...,2019,net primary production;aboveground production;...,"Anderson, John",2003-10-14,2018-09-18,doi:10.6073/pasta/69ca7ac7254a2b236d3f44b21fa8...,210011005,011,signature,Pp,,2003-10-14 to 2018-09-18,1


## Special cases (on Deb's orders)

In [61]:
t1.creators.loc[t1.pkgid.isin([210011001, 210011002, 210011003, 210011004, 210287001])] = 'Peters, Debra; Huenneke, Laura'
t1.creators.loc[t1.pkgid.isin([210002001])] = 'Peters, Debra; Anderson, John; Huenneke, Laura'
t1.creators.loc[t1.pkgid.isin([210121005])] = 'Peters, Debra; Anderson, John; Rango, Al; Huenneke, Laura'
t1.creators.loc[t1.pkgid.isin([210308005])] = 'Peters, Debra; Herrick, Jeff; Okin, Gregory S'
t1.creators.loc[t1.pkgid.isin([210351003])] = 'Bestelmeyer, Brandon; Yao, Jin'
# Append symbol to SMES and similar packages (multiyear=asterisk or complete=dagger)
multiyr = [210086002,210086004,210086005,210086007,210120001,210120002]
t1.title.loc[t1.pkgid.isin(multiyr)] = t1.title.loc[t1.pkgid.isin(multiyr)] + '*'
#complete = [210086001,210086003,210086008,210086009]
#t1.title.loc[t1.pkgid.isin(complete)] = t1.title.loc[t1.pkgid.isin(complete)] + u'\u2020'

# Just drop the most useful NPP package from the table (Sigh)
# and the deprecated soil water
t1 = t1.drop([210011001,210013002])
t1.head()

,packageid,title,pubdate,keyword,creators,begindate,enddate,doi,pkgid,studyid,core_sig_2020prop,corearea,top10_2020prop,drange,corearea_sort
pkgid,,,,,,,,,,,,,,,
210011002,knb-lter-jrn.210011002,Seasonal non-destructive vegetation measuremen...,2019,net primary production;aboveground production;...,"Peters, Debra; Huenneke, Laura",1989-05-01,2018-09-14,doi:10.6073/pasta/9053217dd75ad98e495bef87b459...,210011002,011,signature,Pp,1,1989-05-01 to 2018-09-14,1
210011003,knb-lter-jrn.210011003,Annual mean estimates of aboveground net prima...,2020,net primary production;aboveground production;...,"Peters, Debra; Huenneke, Laura",1990-01-01,2018-12-31,doi:10.6073/pasta/18dad6748af96c98b72cea3436bf...,210011003,011,signature,Pp,1,1990-01-01 to 2018-12-31,1
210011004,knb-lter-jrn.210011004,Seasonal reference harvest measurements of veg...,2019,net primary production;aboveground production;...,"Peters, Debra; Huenneke, Laura",1989-04-25,2018-09-10,doi:10.6073/pasta/bb54f668824c165818255dfa9e1f...,210011004,011,signature,Pp,1,1989-04-25 to 2018-09-10,1
210011005,knb-lter-jrn.210011005,Annual ground-based photographs taken at 15 ne...,2019,net primary production;aboveground production;...,"Anderson, John",2003-10-14,2018-09-18,doi:10.6073/pasta/69ca7ac7254a2b236d3f44b21fa8...,210011005,011,signature,Pp,,2003-10-14 to 2018-09-18,1
210262004,knb-lter-jrn.210262004,Annual Aboveground Net Primary Productivity by...,2020,net primary production;aboveground production;...,"Bestelmeyer, Brandon",2006-05-19,2018-10-31,doi:10.6073/pasta/e5ca11c385ca786e789431baa8d5...,210262004,262,signature,Pp,1,2006-05-19 to 2018-10-31,1


In [62]:
t1_out = t1.loc[:, ['creators','pubdate','title','packageid','corearea','drange',
                    'core_sig_2020prop','top10_2020prop']]
t1_out.to_excel('tables_out/Table1_draft.xlsx', index=False)